<a href="https://colab.research.google.com/github/jungeun919/Pytorch_study/blob/main/computer_vision_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from typing import Tuple, Sequence, Callable
import csv
import cv2
import pandas as pd
from PIL import Image
import torch
import torch.optim as optim
from torch import nn, Tensor
from torch.utils.data import Dataset, DataLoader
# from torchinfo import summary -> pip install torchinfo 후 사용
from torchsummary import summary

from torchvision import transforms
from torchvision.models import resnet50

# 커스텀 데이터셋 만들기

In [2]:
class MnistDataset(Dataset):
    def __init__(
        self,
        dir: os.PathLike,
        image_ids: os.PathLike,
        transforms: Sequence[Callable]
    ) -> None:
        self.dir = dir
        self.transforms = transforms

        self.labels = {}
        with open(image_ids, 'r') as f:
            reader = csv.reader(f)
            next(reader)
            for row in reader:
                self.labels[int(row[0])] = list(map(int, row[1:]))

        self.image_ids = list(self.labels.keys())

    def __len__(self) -> int:
        return len(self.image_ids)

    def __getitem__(self, index: int) -> Tuple[Tensor]:
        image_id = self.image_ids[index]
        image = Image.open(
            os.path.join(self.dir, f'{str(image_id).zfill(5)}.png')).convert('RGB')
        target = np.array(self.labels.get(image_id)).astype(np.float32)

        if self.transforms is not None:
            image = self.transforms(image)

        return image, target

# 이미지 augmentation(증가)

In [3]:
transforms_train = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
])

transforms_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
])

In [4]:
!unzip -qq /content/drive/My\ Drive/Colab\ Notebooks/computer_vision/data_2/dirty_mnist.zip
!unzip -qq /content/drive/My\ Drive/Colab\ Notebooks/computer_vision/data_2/mnist_data.zip
!unzip -qq /content/drive/My\ Drive/Colab\ Notebooks/computer_vision/data_2/test_dirty_mnist.zip

trainset = MnistDataset('/content', '/content/drive/MyDrive/Colab Notebooks/computer_vision/data_2/dirty_mnist_answer.csv', transforms_train)
testset = MnistDataset('/content', '/content/drive/MyDrive/Colab Notebooks/computer_vision/data_2/sample_submission.csv', transforms_test)

# train_loader = DataLoader(trainset, batch_size=256, num_workers=8)
train_loader = DataLoader(trainset,batch_size=32, num_workers=2)

# test_loader = DataLoader(testset, batch_size=32, num_workers=4)
test_loader = DataLoader(testset, batch_size=10, num_workers=1)

# ResNet50 모형

In [5]:
class MnistModel(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.resnet = resnet50(pretrained=True)
        self.classifier = nn.Linear(1000, 26)

    def forward(self, x):
        x = self.resnet(x)
        x = self.classifier(x)

        return x

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MnistModel().to(device)
print(summary(model, (3, 256, 256)))

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth



----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 128, 128]           9,408
       BatchNorm2d-2         [-1, 64, 128, 128]             128
              ReLU-3         [-1, 64, 128, 128]               0
         MaxPool2d-4           [-1, 64, 64, 64]               0
            Conv2d-5           [-1, 64, 64, 64]           4,096
       BatchNorm2d-6           [-1, 64, 64, 64]             128
              ReLU-7           [-1, 64, 64, 64]               0
            Conv2d-8           [-1, 64, 64, 64]          36,864
       BatchNorm2d-9           [-1, 64, 64, 64]             128
             ReLU-10           [-1, 64, 64, 64]               0
           Conv2d-11          [-1, 256, 64, 64]          16,384
      BatchNorm2d-12          [-1, 256, 64, 64]             512
           Conv2d-13          [-1, 256, 64, 64]          16,384
      BatchNorm2d-14          [-1, 256

# 학습하기

In [6]:
import numpy as np

optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MultiLabelSoftMarginLoss()

num_epochs = 10
model.train()
for epoch in range(num_epochs):
    for i, (images, targets) in enumerate(train_loader):
        optimizer.zero_grad()

        images = images.to(device)
        targets = targets.to(device)

        outputs = model(images)
        loss = criterion(outputs, targets)

        loss.backward()
        optimizer.step()

        if (i+1) % 500 == 0:
            outputs = outputs > 0.5
            acc = (outputs == targets).float().mean()
            print(f'{epoch}: {loss.item():.5f}, {acc.item():.5f}')

0: 0.68789, 0.53486
0: 0.68101, 0.55048
0: 0.66473, 0.58413
1: 0.61552, 0.61659
1: 0.65957, 0.58774
1: 0.60738, 0.64663
2: 0.56830, 0.65144
2: 0.60195, 0.64303
2: 0.53248, 0.69591
3: 0.46923, 0.73918
3: 0.49602, 0.74279
3: 0.42080, 0.79087
4: 0.36366, 0.82091
4: 0.40290, 0.81490
4: 0.32814, 0.84615
5: 0.26476, 0.87620
5: 0.28286, 0.89183
5: 0.26483, 0.87620
6: 0.21428, 0.90745
6: 0.20197, 0.91707
6: 0.15565, 0.93149
7: 0.13518, 0.95072
7: 0.12108, 0.95553
7: 0.10118, 0.96034
8: 0.09415, 0.96394
8: 0.11854, 0.95313
8: 0.09283, 0.95553
9: 0.10232, 0.96034
9: 0.06333, 0.97596
9: 0.08050, 0.97236


# 추론하기

In [12]:
submit = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/computer_vision/data_2/sample_submission.csv')

model.eval()
batch_size = test_loader.batch_size
batch_index = 0
for i, (images, targets) in enumerate(test_loader):
    images = images.to(device)
    targets = targets.to(device)
    outputs = model(images)
    outputs = outputs > 0.5
    batch_index = i * batch_size
    submit.iloc[batch_index:batch_index+batch_size, 1:] = \
        outputs.long().squeeze(0).detach().cpu().numpy()

submit.to_csv('/content/drive/MyDrive/Colab Notebooks/computer_vision/data_2/submission1.csv', index=False)